In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import json
with open('gdrive/MyDrive/contacts.json') as f:
  data = json.loads(f.read())

In [3]:
data[0:3]

[{'Contacts': 1,
  'Email': 'gkzAbIy@qq.com',
  'Id': 0,
  'OrderId': '',
  'Phone': ''},
 {'Contacts': 4,
  'Email': '',
  'Id': 1,
  'OrderId': 'vDDJJcxfLtSfkooPhbYnJdxov',
  'Phone': '329442681752'},
 {'Contacts': 0, 'Email': '', 'Id': 2, 'OrderId': '', 'Phone': '9125983679'}]

In [4]:
import pandas as pd
df = pd.json_normalize(data)

In [5]:
df.head()

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,,1,
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,,9125983679,0,
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,,300364407,2,


In [6]:
print(len(df[(df.Email != "") | (df.Phone != "") | (df.OrderId != "")]))
print(df.Email.nunique())
print(df.Phone.nunique())
print(df.OrderId.nunique())

500000
249157
190678
189303


In [7]:
# Union-Find Structrue
class UF:
  def __init__(self, node_list):
    self.data = {nd: nd for nd in node_list}

  def _find_deepest_node(self, node):
    if self.data[node] != node:
      self.data[node] = self._find_deepest_node(self.data[node])
      return self.data[node]
    else:
      return node

  def connect(self, node_1, node_2):
    if node_1 > node_2: node_1, node_2 = node_2, node_1
    self.data[self._find_deepest_node(node_2)] = self._find_deepest_node(node_1)

  def get_data(self):
    return {k: self._find_deepest_node(v) for k, v in self.data.items()}

In [8]:
# Test UF structure
tmp_uf = UF([3, 5, 6, 42, 100, 201])
print("original data:")
print(tmp_uf.data, "\n")
tmp_uf.connect(6, 3)
print("connect 6, 3: \n", tmp_uf.data, "\n", sep="")
tmp_uf.connect(201, 42)
print("connect 201, 42: \n", tmp_uf.data, "\n", sep="")
tmp_uf.connect(42, 6)
print("connect 42, 6: \n", tmp_uf.data, "\n", sep="")
print("final (pretiffied) data:")
print(tmp_uf.get_data())

original data:
{3: 3, 5: 5, 6: 6, 42: 42, 100: 100, 201: 201} 

connect 6, 3: 
{3: 3, 5: 5, 6: 3, 42: 42, 100: 100, 201: 201}

connect 201, 42: 
{3: 3, 5: 5, 6: 3, 42: 42, 100: 100, 201: 42}

connect 42, 6: 
{3: 3, 5: 5, 6: 3, 42: 3, 100: 100, 201: 42}

final (pretiffied) data:
{3: 3, 5: 5, 6: 3, 42: 3, 100: 100, 201: 3}


In [9]:
# UF initialization
node_list = [x['Id'] for x in data]
assert len(node_list) == len(set(node_list)) # make sure ids unique
uf = UF(node_list)

In [10]:
for feat in ['OrderId', 'Phone', 'Email']:
  val_cnt = df[feat].value_counts()
  candidate = {x for x in val_cnt[(val_cnt > 1)].index if x != ""}
  for _, grp in df[df[feat].isin(candidate)].groupby(feat):
    group_ids = set(grp.Id)
    pivot_id = min(group_ids)
    for id_ in group_ids:
      if id_ != pivot_id:
        uf.connect(pivot_id, id_)

In [11]:
# id -> group id
remap = uf.get_data()

In [12]:
# sanity check
for i in sorted({215197, 50, 404324, 5, 212533, 226720, 458692, 383605, 482810}):
  print(f'{i:>10}:    {remap[i]}')

         5:    404324
        50:    404324
    212533:    404324
    215197:    404324
    226720:    404324
    383605:    404324
    404324:    404324
    458692:    404324
    482810:    404324


In [13]:
# Process data frame according to group id
df['groupId'] = [remap[id_] for id_ in df.Id]
df = df.assign(newId=df.groupby('groupId')['Id'].transform(lambda x: '-'.join(str(s) for s in sorted(x, key=int))))
df = df.assign(ContactsSum=df.groupby('groupId')['Contacts'].transform('sum'))
df.head(5)

,Id,Email,Phone,Contacts,OrderId,groupId,newId,ContactsSum
0,0,gkzAbIy@qq.com,,1,,0,0,1
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,140081,1-2458-98519-115061-140081-165605-476346,12
2,2,,9125983679,0,,159312,2-159312-322639-348955,4
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa,3,3,0
4,4,,300364407,2,,4,4,2


In [14]:
# sanity check
print(df[df.Id.isin({215197, 50, 404324, 5, 212533, 226720, 458692, 383605, 482810})].Contacts.sum())
df[df.Id.isin({215197, 50, 404324, 5, 212533, 226720, 458692, 383605, 482810})]

15


,Id,Email,Phone,Contacts,OrderId,groupId,newId,ContactsSum
5,5,,840113148,0,,404324,5-50-212533-215197-226720-383605-404324-458692...,15
50,50,TIMejlVYhfqjy@gmail.com,840113148,4,XBkNrwcWWslDvSQdhYPfCpQJo,404324,5-50-212533-215197-226720-383605-404324-458692...,15
212533,212533,izafiOnTIVibbz@hotmail.com,04962058234,0,,404324,5-50-212533-215197-226720-383605-404324-458692...,15
215197,215197,izafiOnTIVibbz@hotmail.com,2529522887,0,sEgewZTtnDMAADcwLSjxsHiSK,404324,5-50-212533-215197-226720-383605-404324-458692...,15
226720,226720,,2529522887,4,XBkNrwcWWslDvSQdhYPfCpQJo,404324,5-50-212533-215197-226720-383605-404324-458692...,15
383605,383605,izafiOnTIVibbz@hotmail.com,,0,,404324,5-50-212533-215197-226720-383605-404324-458692...,15
404324,404324,yFcCGIfYcFKfmqpNk@yahoo.com,,3,,404324,5-50-212533-215197-226720-383605-404324-458692...,15
458692,458692,yFcCGIfYcFKfmqpNk@yahoo.com,,0,XBkNrwcWWslDvSQdhYPfCpQJo,404324,5-50-212533-215197-226720-383605-404324-458692...,15
482810,482810,yFcCGIfYcFKfmqpNk@yahoo.com,,4,,404324,5-50-212533-215197-226720-383605-404324-458692...,15


In [15]:
# submission
sub_df = df[['Id', 'newId']].copy()
sub_df['newId'] = sub_df['newId'] + ", " + df['ContactsSum'].astype(str)
sub_df.columns = ['ticket_id', 'ticket_trace/contact']
sub_df = sub_df.sort_values('ticket_id').reset_index(drop=True)
sub_df.head()

,ticket_id,ticket_trace/contact
0,0,"0, 1"
1,1,"1-2458-98519-115061-140081-165605-476346, 12"
2,2,"2-159312-322639-348955, 4"
3,3,"3, 0"
4,4,"4, 2"


In [16]:
sub_df.to_csv("answer.csv", index=False)